In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase, str_rep_vdb_ing, str_rep_vdb_nam 
import pandas as pd
additional_cbr_1, additional_cbr_2, additional_validation, cbr_augmentation, cbr_database, model_training, validation = load_all_datasets()

DATABASE_NAME = "cbr_database"
OUTPUT_FILE = "cbr_augmentation_gpt2_input.csv"
cbr_id = cbr_database.set_index("id")

In [2]:
# large_cbr = pd.concat([cbr_database, cbr_augmentation, additional_cbr_1, additional_cbr_2])
# large_cbr_id = large_cbr.set_index("id")
# large_cbr_reps = str_rep_vdb(large_cbr_id)
# len(large_cbr_reps)

In [3]:
# meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_id["id"].values)], {"dataset":"cbr_database"})
# vdb = CBRDatabase(large_cbr_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)

vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)
print(len(vdb.get_texts()))
print(vdb.similarity_search(query="test", k=1))

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
10627
[Document(page_content='name: ww 2 points potato pepper and egg scramble ; ingredients: olive oil, bell peppers, onion, baking potato, fat free egg substitute, parsley, salt, fresh black pepper ; preparation:', metadata={'dataset': 'cbr_database', 'recipe_id': 250756})]


In [4]:
# generowanie inputów - 1500 tokenów 
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in cbr_augmentation.iterrows():
    while True:
        # recipe_ixs = [random.randint(0, len(cbr_database)-1) for _ in range(7)]
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=5)]
        # recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query=v_recipe["ingredients"], k=7)]
        # recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query=v_recipe["name"], k=7)]
        # print(vdb.similarity_search(query=v_recipe["ingredients"], k=7))#dev
        # print(large_cbr_id.loc[recipe_ixs])#dev

        gen_input = compose(cbr_id.loc[recipe_ixs], v_recipe)
        br = False
        if len(tokenizer.encode(gen_input)) <= 850: 
            br = True
        else:
            for ii in list(range(6, -1, -1)):
                gen_input = compose(cbr_id.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(5, -1, -1)):
                gen_input = compose(cbr_id.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_id.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
        
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2 - 654
1 - 732
1 - 761
1 - 681
1 - 773
1 - 681
1 - 763
1 - 790
1 - 776
1 - 660
1 - 833
1 - 812
1 - 740
1 - 765
1 - 638
1 - 731
1 - 837
1 - 776
1 - 728
1 - 801
1 - 773
1 - 822
1 - 749
1 - 795
1 - 778
1 - 644
1 - 759
1 - 764
1 - 762
1 - 744
1 - 742
1 - 784
1 - 848
1 - 828
1 - 719
1 - 726
1 - 756
1 - 811
1 - 780
1 - 664
1 - 751
1 - 681
1 - 770
1 - 729
1 - 688
1 - 770
1 - 754
1 - 655
1 - 785
1 - 816
1 - 837
1 - 736
1 - 669
1 - 690
1 - 743
1 - 753
1 - 825
1 - 802
1 - 742
1 - 664
1 - 692
1 - 693
1 - 695
1 - 741
1 - 730
1 - 793
2 - 683
1 - 833
1 - 804
1 - 704
1 - 806
1 - 821
1 - 742
1 - 786
1 - 747
1 - 848
1 - 776
1 - 720
1 - 668
1 - 776
1 - 692
1 - 795
1 - 803
1 - 774
1 - 697
1 - 804
1 - 761
1 - 711
1 - 708
1 - 701
1 - 779
1 - 801
1 - 721
1 - 773
1 - 796
1 - 845
1 - 775
1 - 796
1 - 805
1 - 775
1 - 713
1 - 694
1 - 737
1 - 769
1 - 850
1 - 680
1 - 836
1 - 709
1 - 844
1 - 820
1 - 834
1 - 845
1 - 743
1 - 756
1 - 703
1 - 805
1 - 713
2 - 670
1 - 849
1 - 759
1 - 675
1 - 714
1 - 757
1 - 743
1 - 627


In [5]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}", index=False)

In [6]:
vdb.similarity_search(query="str_rep_no_steps(v_recipe)", k=7)

[Document(page_content='name: v s crock pot bottom round roast ; ingredients: boneless bottom round roast, dijon mustard, brown sugar, worcestershire sauce, dry red wine, beef broth, onion, carrots, kosher salt & freshly ground black pepper ; preparation:', metadata={'dataset': 'cbr_database', 'recipe_id': 276903}),
 Document(page_content='name: holiday vegan golden gravy ; ingredients: flour, nutritional yeast flakes, oil, water, soy sauce, salt & pepper ; preparation:', metadata={'dataset': 'cbr_database', 'recipe_id': 35838}),
 Document(page_content='name: savory slow cooker beef roast ; ingredients: boneless beef roast, onion, vegetable juice, brown gravy mix, garlic cloves, pepper, thyme ; preparation:', metadata={'dataset': 'cbr_database', 'recipe_id': 257081}),
 Document(page_content='name: vegan caesar salad dressing ; ingredients: dijon mustard, nutritional yeast flakes, almonds, garlic cloves, water, lemon juice, soy sauce, olive oil, white pepper ; preparation:', metadata={'

In [7]:
from collections import Counter


Counter(df.input.apply(lambda x: x.split("name:")).apply(lambda x: len(x)))

Counter({6: 5461, 7: 5002, 5: 164})